- [x] Using the titanic data, in your classification-exercises repository, create a notebook, decision_tree.ipynb where you will do the following:


In [39]:
# Import full libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import personal modules
import acquire as a
import prepare as p
import model as m

# Selective imports
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Handle warnings
import warnings
warnings.filterwarnings("ignore")

In [49]:
# Load titanic
titanic_raw = a.get_titanic_data()
titanic_raw.head()

Reading from file...


,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [66]:
# Prepare titanic
titanic_prepped = p.prep_titanic(titanic_raw)
titanic_prepped.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,Southampton,1


In [73]:
# make sure dataset is fully clean
a.df_info(titanic_prepped)

,nunique,dtypes,isnull,sample
passenger_id,891,object,0,880
survived,2,object,0,1
pclass,3,object,0,2
sex,2,object,0,female
age,88,float64,177,25.0
sibsp,7,object,0,0
parch,7,object,0,1
fare,248,float64,0,26.0
embark_town,3,object,2,Southampton
alone,2,object,0,0


> At this time, I've moved a piece of the function from the preprocess titanic function over to the prep_titanic function to handle the age nulls.

> It's still missing the piece to fix the nulls in embark_town.

In [95]:
# fill based on mode
titanic_prepped['embark_town'] = titanic_prepped.embark_town.fillna(titanic_prepped.embark_town.mode()[0])

In [97]:
# check the data again
a.df_info(titanic_prepped)

,nunique,dtypes,isnull,sample
passenger_id,891,int64,0,152
survived,2,int64,0,0
pclass,3,int64,0,3
sex,2,object,0,male
age,89,object,0,55.5
sibsp,7,int64,0,0
parch,7,int64,0,0
fare,248,float64,0,8.05
embark_town,3,object,0,Southampton
alone,2,int64,0,1


In [55]:
# Split titanic
train_titanic,validate_titanic,test_titanic = p.split_df(titanic_prepped,'survived')
train_titanic.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone
776,776,0,3,male,NaN,0,0,7.7500,Queenstown,1
829,829,1,1,female,62.0,0,0,80.0000,NaN,1
215,215,1,1,female,31.0,1,0,113.2750,Cherbourg,0
258,258,1,1,female,35.0,0,0,512.3292,Cherbourg,1
129,129,0,3,male,45.0,0,0,6.9750,Southampton,1


1. [ ] What is your baseline prediction? What is your baseline accuracy? *remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.*

In [64]:
# Get baseline prediction
train_titanic.survived.value_counts()

survived
0    329
1    205
Name: count, dtype: int64

In [ ]:
# Get baseline accuracy

2. [ ] Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

3. [ ] Evaluate your in-sample results using the model score, confusion matrix, and classification report.

4. [ ] Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

5. [ ] Run through steps 2-4 using a different `max_depth` value.

6. [ ] Which model performs better on your in-sample data?

7. [ ] Which model performs best on your out-of-sample data, the `validate` set?


---

1. [ ] Work through these same exercises using the Telco dataset.

2. [ ] Experiment with this model on other datasets with a higher number of output classes.